# 盘间带降落伞模拟

AERO-Suite里的**AERO-F**是流体仿真模拟器，这里是详细的[**AERO-F**教程](https://bitbucket.org/frg/aero-f/downloads/AERO-F.pdf)。
**AERO-S**是固体仿真模拟器，这里是详细的[**AERO-S**教程](https://bitbucket.org/frg/aero-s/downloads/AERO-S.pdf)。

本案例是使用浸入边界方法对盘间带降落伞的流固耦合的仿真模拟。该应用将使用自适应网格、固体接触碰撞、大窝湍流模型。本次仿真的文件都在 `Parachute_DGB` 文件夹中，
在这个文件中一般有3个子文件夹：`sources`, `simulations`和`data`。
- `sources`文件夹一般存放网格相关的源文件
- `simulations`文件夹一般存放仿真模拟的输入文件
- `data`文件夹一般存放仿真模拟过程中生成的一些文件。

以下是本次仿真模拟的详细步骤。
在开始本次仿真模拟前，我们需要安装相关的软件，请参加安装教程，比如[如何在北京大学未名一号上安装AERO-Suite](../../../Install/Install_PKU.ipynb)。
如果我们之前已经安装好相关软件，我们可以读取 [.bashrc_frg](../../../Install/bashrc_frg) 刷新当前shell环境，载入各个软件的命令
```shell
source ~/.bashrc_frg 
```

# 网格文件

## 流体网格
由于我们使用的自适应网格算法，最新节点二分法(newest node bisection)，对初始网格的编号有一定的要求。 我们在 `sources` 文件夹中，用**python**生成长方体的计算区域，以及均匀的四面体网格

```shell
python generateMesh.py
```
我们能在 [generateMesh.py](../sources/generateMesh.py) 中的 `parachute3D()` 函数里，指定 `x`, `y`, `z` 数组，并指定`z`底部，`z`顶部，`x`底部，`x`顶部，`y`底部，`y`顶部的边界名称，生成结构网格 `fluid.top`。


## 固体网格
降落伞分为盘状部分和袋装部分，由共80块三角形布条组成，每块三角形布条由悬挂线分隔，在降落伞的盘状、带状两部分边缘也有缝线。
固体网格 [StructureFile.include](../sources/StructureFile.include)在 `sources` 文件夹中，它包含了二维的壳单元和一维的梁单元，每个点的自由度是6。值得注意的是固体网格所表示的降落伞并没有折叠，在固体网格文件末，我们有用关键词 *IDISP6* 描述降落伞当前状态到折叠状态的位移作为初始位移，位移自由度也是6，包括3个位移自由度以及3个旋转自由度。

由于我们要考虑接触碰撞，我们需要指定需要避免碰撞的主面(Master surface)和从面(Slave surface)，它们可以是相同的。我们把降落伞所有壳单元当做了一个面放在 `sources` 文件夹的 [SelfContactSurfaceTopology.include](../sources/SelfContactSurfaceTopology.include) 文件，里面的关键词是 *SURFACETOPO* 后面的数字是编号，这里我们考虑的是一个方向的接触碰撞，这个方向是这些面的外法向，即只有外法向方向要碰撞了才会产生碰撞作用力。当然我们也可以在里面定义很多面，考虑不同面之间的接触碰撞。

我们有一些辅助函数在 `sources/prepro` 文件夹的 [foldParachute.py](../sources/prepro/foldParachute.py) 里面用于计算降落伞的初始位移、加密降落伞网格和生成接触面文件等。

对于用于可视化的固体网格，我们可以选取任意流固耦合仿真的输入文件，比如 `simulations` 文件夹中的 [StructureFile.Folding](./StructureFile.Folding)，用**AERO-S**可以生成相应的固体网格文件 `Structure.top`：

```shell
aeros -t StructureFile.Folding
```


## 浸入表面网格
对于使用浸入边界方法的仿真模拟，我们需要输入流体求解器表示固体形状的浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top)，这个浸入表面可以和固体表面网格相同，也能与之不同。本案例中，浸入表面表示了降落伞、以及探测器、六边形横截面的悬挂线(参见[软管的流固耦合模拟](../../Beam/simulations/BEAM_EBM_FSI_README.ipynb))。固体网格只包含了降落伞和用梁单元建模的悬挂线，因而他们并不相同。在`sources/prepro`文件夹，我们可以用

```shell
gmsh -3 capsule.geo -format msh22
gmsh2top capsule
```
生成探测器网格 `capsule.top`， 其中 `capsule.geo` 是用**gmsh**做的探测器形状。

在`sources/prepro`文件夹里有把固体网格[StructureFile.include](../sources/StructureFile.include)删去一部分处理过后的固体文件，其中 `parachute_emb_mesh.top` 文件有所有的顶点，但是有限元只包括降落伞部分，`parachute_susp_line_emb_mesh.top` 文件有所有的顶点，但是有限元只包括降落伞和悬挂线。我们使用**python**文件`computeEmbeddedSurf.py` 读入这些处理后的固体文件和探测器网格生成浸入表面网格。在`computeEmbeddedSurf.py`中，我们可以设置我们的流固耦合模拟是否考虑探测器(considerCapsule），是否考虑流体和悬挂线的相互作用(considerSuspensionLine)。如果考虑流体和悬挂线的相互作用，我们的浸入表面网格里就会有悬挂线。对于当前模拟，我们考虑了探测器但是忽略了流体和悬挂线的相互作用。
值得注意，在表示固体形状的浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top) 文件里，我们有3个 *StickMovingSurface_* 分布是降落伞的盘状、带状部分和探测器，这些边界默认的边界条件是*Wall*边界条件。我们可以用这些编号来指定不同的边界条件，比如透气的边界条件（*PorousWall*）。

```shell
python computeEmbeddedSurf.py
```
会生成 `embeddedSurface.top`。
最后我们需要使用一下命令把 `capsule.top` 和 `embeddedSurface.top` 放进`sources`文件夹
```shell
mv capsule.top  ../.
mv embeddedSurface.top ../.
```


我们可以用**xp2exo**把 `.top` 网格文件转化为 `.exo` 文件进行可视化，这里我们展示了流体网格、浸入表面、和固体网格:
<img src="../../Figs/PID_Mesh.png" width="800" />


# 网格前处理


对于网格的前处理，我们可以把这些命令行写在 `Parachute_DGB` 文件夹里的脚本文件里，比如[preprocess.sh](../preprocess.sh)。我们可以用以下命令行运行脚本

```shell
bash preprocess.sh
```

## 流体部分
在这里面我们使用**partnmesh**软件，能把流体网格划分200个区域，使用200个核进行并行运算。
使用**matcher**去匹配浸入表面 [embeddedSurface.top](../sources/embeddedSurface.top) 里标注了是 *Moving* 的边界单元的格点（比如StickMovingSurface），和固体表面的网格 [matcher.top](../sources/matcher.top)。如果考虑了悬挂线，我们在匹配梁单元时，需要加命令行 `-beam`。
之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在`data`文件夹中，以 `fluidmodel` 为前缀。

## 固体部分
对于固体部分，我们也需要并行，在[空心圆柱的固体模拟](../../Cylinder/simulations/CYLINDER_README.ipynb)中我们介绍了固体的并行网格划分。 我们把固体网格划分96个区域，使用96个核进行并行运算。
对于本应用，我们采用了Philip Avery 设计的用 `sources/prepro`文件夹里 `dec.cpp` 和 `mod.cpp`，手动对降落伞进行划分的方法，把降落伞相邻的布块放在了一组，共80组，然后把悬挂线等分成了16组，共96个分区，分区结果手动存放在 `SowerFile.optDec`里，其中每个分区先存放有限元数量然后存放的是有限元编号。 最初，这样分区是为了加速接触碰撞的，使得我们只考虑相同分区里的碰撞，但是最后我们并没有这样使用，而是考虑了全部的单向的碰撞。当然我们也可以用 **aeros** 进行分区

```shell
aeros --dec --nsub 96 sources/SowerFile
```

自动生成分区文件 `SowerFile.optDec`。之后我们需要用**sower**去生成各个区域网格的详细信息，把这些信息文件储存在`data`文件夹中，以 `structuremodel` 为前缀。

# 流固耦合计算
本次案例的流固耦合仿真计算总共包括以下7个仿真计算
- 降落伞折叠(Folding)：对降落伞进行折叠，能考虑折叠后的应力
```shell
sbatch Sbatch.Folding.sh
bash preprocess.Folding.sh
```
- 流体网格预加密(APriori)：对指定的求解区域进行网格预加密
```shell
sbatch Sbatch.APriori.sh
bash preprocess.APriori.sh
```
- 探测器流体模拟(Capsule)：假设降落伞还没有弹出，对只有探测器的流体做模拟仿真，生成流固耦合模拟仿真中流体的初始状态
```shell
sbatch Sbatch.Capsule.sh
bash preprocess.Capsule.sh
vim references.Capsule/Restart.data  #只保留 MaxNodePreAMR = ...
```
- 降落伞弹出模拟(Pingpong)：降落伞弹出，处于半折叠状态，生成半折叠状态的浸入表面网格 
```shell
sbatch Sbatch.Pingpong.sh
```
- 半折叠降落伞流体模拟(Unsteady)：假设固体是不动的，对流体做模拟仿真，生成流固耦合模拟仿真中流体的初始状态
```shell
sbatch Sbatch.Unsteady.sh
bash preprocess.Unsteady.sh
vim references.Unsteady/Restart.data  #只保留 MaxNodePreAMR = ...
```
- 流固耦合模拟(FSI)
```shell
sbatch Sbatch.FSI.sh
```
- 流固耦合模拟重启(FSI.Restart)
```shell
sbatch Sbatch.FSI.Restart.sh
bash preprocess.FSI.sh
```

## 降落伞折叠 (Folding)

我们对降落伞进行折叠，之后的模拟用折叠后的降落伞作为初始状态，能折叠后的应力。 我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.Folding.sh
```
这会分配96个CPU核运行 `run.Folding.sh` 里的固体模拟。降落伞的折叠用初始位移表示IDISP6，因此只需要计算一个时间步就能得到折叠后的降落伞，折叠后的降落伞存在重启文件中 `references.Folding/RESTART_FEM`。最后，我们可以在`simulations`文件夹里用
```shell
bash postprocess.Folding.sh
```
对结果进行后处理，折叠后的降落伞如下图所示。
<img src="../../Figs/PID_DGB_Folding.png" width="300" />

## 流体网格预加密 (APriori)

我们对指定的流体求解区域进行网格预加密，之后不能粗化这些网格。这里我们使用的浸入边界只包含探测器，我们对探测器周围的边界层 (探测器固定了)，降落伞、和探测器周围和尾流位置进行了加密。 
我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.APriori.sh
```
这会分配200个CPU核运行 `run.APriori.sh` 里的流体模拟。加密后的网格信息存放在 `references.APriori`， 之后我们用加密后的网格作为初始网格进行模拟。最后，我们可以在`simulations`文件夹里用
```shell
bash postprocess.APriori.sh
```
对结果进行后处理，预加密后的流体网格如下图所示。
<img src="../../Figs/PID_DGB_APriori.png" width="300" />



## 探测器流体模拟 (Capsule)

假设降落伞还没有弹出，对只有探测器的流体做模拟仿真，生成流固耦合模拟仿真中流体的初始状态，这里流体网格，
我们使用存放在 `references.APriori` 预加密后的网格。
我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.Capsule.sh
```
这会分配200个CPU核运行 `run.Capsule.sh` 里的流体模拟。运行后用于之后仿真的重启文件存放在 `references.Capsule`， 
之后我们在此基础上继续模拟。最后，我们可以在`simulations`文件夹里用
```shell
bash postprocess.Capsule.sh
```
对结果进行后处理，折叠后的降落伞如下图所示。
<img src="../../Figs/PID_DGB_Capsule.png" width="400" />

注意：对于存放在 `references.Capsule` 文件夹中的重启数据文件 `Restart.data`， 只保留 *RestartParameters* 
里面的 *MaxNodePreAMR* 信息，删除其它所有信息。因为重新开始模拟需要保证不粗化编号在 *MaxNodePreAMR* 前面的顶点。



## 降落伞弹出模拟 (Pingpong)：

降落伞弹出，处于半折叠状态，这个模拟只为了生成半折叠状态的浸入表面网格。
存放在 `references.Pingpong` 文件夹里`EmbeddedPosition.data`， 这个文件记录了浸入表面网格位移，用于重启。 
我们可以在`simulations`文件夹运行以下脚本文件

```shell
sbatch Sbatch.Pingpong.sh
```

这会分配296个CPU核运行 `run.Pingpong.sh` 里的流固耦合模拟。我们近需要上述浸入表面网格重启文件。


## 半折叠降落伞流体模拟 (Unsteady)

假设降落伞弹出，对半折叠降落伞（固定降落伞）进行流体模拟仿真，生成流固耦合模拟仿真中流体的初始状态，
这里我们用探测器流体模拟(Capsule)的结果做流体的初始状态，用降落伞弹出模拟(Pingpong)的结果做浸入边界网格（固定降落伞）的初始状态。
我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.Unsteady.sh
```
这会分配200个CPU核运行 `run.Unsteady.sh` 里的流体模拟。运行后用于之后仿真的重启文件存放在 `references.Unsteady`， 
之后我们在此基础上继续模拟。最后，我们可以在`simulations`文件夹里用
```shell
bash postprocess.Unsteady.sh
```
对结果进行后处理，半折叠降落伞流体模拟如下图所示。
<img src="../../Figs/PID_DGB_Unsteady.png" width="300" />

注意：对于存放在 `references.Unsteady` 文件夹中的重启数据文件 `Restart.data`， 只保留 *RestartParameters* 
里面的 *MaxNodePreAMR* 信息，删除其它所有信息。因为重新开始模拟需要保证不粗化编号在 *MaxNodePreAMR* 前面的顶点。


## 流固耦合模拟(FSI)

我们用半折叠降落伞流体模拟 (Unsteady)的结果做流体的初始状态，用降落伞弹出模拟(Pingpong)的结果做浸入边界网格（固定降落伞）的初始状态。
我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.FSI.sh
```
这会分配496个CPU核运行 `run.FSI.sh` 里的流固耦合模拟。运行后用于之后仿真的重启文件存放在 `references.FSI`， 

## 流固耦合模拟重启(FSI.Restart)


由于集群的时间限制，我们仿真还没有结束，我们需要重启模拟，我们可以在`simulations`文件夹运行以下脚本文件
```shell
sbatch Sbatch.FSI.Restart.sh
```
这会分配496个CPU核运行 `run.FSI.Restart.sh` 里的流固耦合模拟。


最后，我们可以在`simulations`文件夹里用
```shell
bash postprocess.FSI.sh
```
对结果进行后处理，半折叠降落伞流体模拟如下图所示。
<img src="../../Figs/PID_DGB_FSI.png" width="300" />
